# LangChain 1.X.X+ – nowa składnia: create_agent, messages, structured output, pamięć, middleware, streaming, MCP

Ten notebook jest **dodatkowym uzupełnieniem** do kursu – pokazuje kilka kluczowych elementów z nowej składni LangChain (1.x), w szczególności API wokół `create_agent`.
Kurs w wielu miejscach wykorzystuje przykłady z uzyciem langchain_classic, które mogą zostać zastąpione przez uproszconą składnię LangChain w wersji powyżej 1.x.

### Instalacja bibliotek

In [ ]:
!pip install -q langchain python-dotenv langchain_mcp_adapters fastmcp

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### create_agent

W nowej składni agent jest tworzony przez `create_agent(model=..., tools=[...], ...)` i wywoływany przez `invoke()`.

In [2]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Rate the city."""
    return f"{city} is the best place in the world!"

agent = create_agent(
    model="gpt-5-mini",
    tools=[rate_city],
    system_prompt="You are a helpful assistant",
)

result = agent.invoke({"messages": [{"role": "user", "content": "Is Poznań a nice city?."}]})

last_msg = result["messages"][-1]
print(last_msg.content)


Short answer: yes — Poznań is generally considered a very nice city, especially for history, culture, students and a relaxed, lived-in Polish city vibe.

Reasons people like Poznań
- Beautiful Old Town (Stary Rynek) with the Renaissance Town Hall and the famous mechanical goats that butt heads at noon.
- Rich history: Cathedral Island (Ostrów Tumski), churches, and museums (National Museum, Archeological Museum).
- Green space: Lake Malta (recreation, rowing, walking), Citadel Park (large park, monuments, views).
- Vibrant cultural scene: Malta Festival, theatres, concerts and active contemporary arts.
- Student energy: Adam Mickiewicz University gives the city a lively cafe/bar/nightlife scene and affordable cultural offerings.
- Food and local specialties: Poznań’s St. Martin’s croissant (rogale świętomarcińskie), good cafes and craft beer.
- Good transport links: international airport (Poznań–Ławica), rail connections to Warsaw, Berlin and other cities; compact city centre that's ea

### Messages – obiekty wiadomości

LangChain standaryzuje wiadomości jako obiekty (`SystemMessage`, `HumanMessage`, `AIMessage`, `ToolMessage`) i pozwala używać ich z modelami czatu.

In [3]:
from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage, HumanMessage

chat = init_chat_model("gpt-5-mini")

messages = [
    SystemMessage("You are a concise assistant."),
    HumanMessage("Write a 1-sentence summary of what LangChain is."),
]

ai_msg = chat.invoke(messages)  # -> AIMessage
print(ai_msg.content)

LangChain is an open-source framework for building applications with large language models by composing chains, prompts, agents, memory, and integrations (e.g., vector stores, tools, and external data) to connect LLMs to real-world workflows.


### Structured output – `response_format` w `create_agent`

Zamiast parsować tekst, możesz poprosić agenta o zwrócenie danych zgodnych z typem (np. Pydantic model).
Efekt trafia wtedy do `result["structured_response"]`.


In [4]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address")
    phone: str = Field(description="The phone number")

agent = create_agent(
    model="gpt-5-mini",
    response_format=ContactInfo,
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}
    ]
})

structured = result["structured_response"]
print(structured)
print(type(structured))


name='John Doe' email='john@example.com' phone='(555) 123-4567'
<class '__main__.ContactInfo'>


### Short‑term memory

W LangChain 1.X.X pamięć *krótkoterminowa* jest realizowana przez dedykowany komponent InMemorySaver.

- `InMemorySaver()` trzyma stan w pamięci procesu (idealne do notebooka)
- `thread_id` identyfikuje „wątek rozmowy” (ważne przy wielokrotnych `invoke()`)


In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    checkpointer=checkpointer,
)

config = {"configurable": {"thread_id": "demo-thread-1"}}

agent.invoke({"messages": [{"role": "user", "content": "Hi! My name is Michał."}]}, config=config)
result = agent.invoke({"messages": [{"role": "user", "content": "What is my name?"}]}, config=config)
print(result["messages"][-1].content)


Your name is Michał.


### Middleware - Human‑in‑the‑loop middleware

Middleware pozwala „wpiąć się” w wykonanie agenta.
Typowy scenariusz: agent może *odczytywać* dane, ale akcje typu *wysyłka e‑maila / zapis do bazy* wymagają akceptacji.
Human‑in‑the‑loop – zatrzymanie przed uruchomieniem wybranych narzędzi i wznowienie przez `Command(resume=...)`.


In [6]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

def read_email(email_id: str) -> str:
    """Read email mock function"""
    return f"(mock) Email content for id={email_id}"

def send_email(recipient: str, subject: str, body: str) -> str:
    """Send email mock function"""
    return f"(mock) Sent email to {recipient} with subject={subject} and content={body}"

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    tools=[read_email, send_email],
    checkpointer=checkpointer,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {"allowed_decisions": ["approve", "edit", "reject"]},
                "read_email": False,
            }
        )
    ],
)

config = {"configurable": {"thread_id": "hitl-demo"}}

paused = agent.invoke(
    {"messages": [{"role": "user", "content": "Send an email to alice@example.com with subject 'Hi' and say hello."}]},
    config=config,
)
print("Paused state keys:", paused.keys())

Paused state keys: dict_keys(['messages', '__interrupt__'])


In [7]:
resumed = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config,
)
print(resumed["messages"][-1].content)

Done — I sent the email to alice@example.com with subject "Hi" and body "Hello."


### Middleware -Guardrails – PII middleware

Jako middelware możemy też skorzystać z gurdrails czyli np. zabezpieczenia przed wyciekiem danych wrażliwych w odpowiedzi LLM.

In [8]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

def echo(text: str) -> str:
    """Print text."""
    return text

agent = create_agent(
    model="gpt-5-mini",
    tools=[echo],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)

out = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Extract information from text: My email is john@example.com and card is 5105-1051-0510-5100"
    }]
})
print(out["messages"][-1].content)


Extracted fields:
- Email: [REDACTED_EMAIL]
- Card (masked): ****-****-****-5100
- Card last 4 digits: 5100

Note: The email appears to be already redacted in the input. If you want a different output format (JSON, CSV, etc.), tell me which.


### Streaming

W LangChain możemy zwracać  częsciowe odpowiedzi lub kolejne wygenerowane tokeny "w locie":
- **postęp agenta** (`stream_mode="updates"`) – event po każdym kroku
- **tokeny/model messages** (`stream_mode="messages"`) – UI-friendly


In [11]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Rate city mock tool."""
    return f"The best city is {city}!"

agent = create_agent(model="gpt-5", tools=[rate_city])

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Is Poznań a nice city? Rate city and afterwards plan a trip to Poznań in 5 stages."}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        last = data["messages"][-1]
        print(f"step: {step:>6} | type={type(last).__name__}")
        try:
            print("content_blocks:", last.content_blocks)
        except Exception:
            print("content:", getattr(last, "content", None))


step:  model | type=AIMessage
content_blocks: [{'type': 'tool_call', 'id': 'call_X2n1l0Y8fCFsQhSzVCzeOOob', 'name': 'rate_city', 'args': {'city': 'Poznań'}}]
step:  tools | type=ToolMessage
content_blocks: [{'type': 'text', 'text': 'The best city is Poznań!'}]
step:  model | type=AIMessage
content_blocks: [{'type': 'text', 'text': 'Rating: The best city is Poznań!\n\n5-stage trip plan to Poznań\n\n1) Old Town essentials (Stare Miasto)\n- Stary Rynek (Main Square) and Renaissance Town Hall; see the mechanical goats at noon.\n- Poznań Fara (Baroque parish church).\n- Climb the Royal Castle tower for views.\n- Try a rogal świętomarciński (St. Martin’s croissant); consider the Rogal Museum show.\n\n2) Cathedral Island and Śródka\n- Ostrów Tumski: Archcathedral Basilica (site tied to Poland’s first rulers).\n- Porta Posnania interactive museum for the city’s origins.\n- Walk over Bishop Jordan Bridge to Śródka for the mural and cozy eateries.\n\n3) Culture and architecture loop\n- Stary Bro

### MCP – Model Context Protocol

MCP standaryzuje wykorzystuje zewnetrznych narzędzi przez LLM.
W LangChain używamy adaptera `langchain-mcp-adapters` i klienta `MultiServerMCPClient`, który potrafi pobrać listę narzędzi z wielu serwerów.


#### Minimalny MCP server (FastMCP)
Zapisz poniższy kod jako `math_server.py` w tym samym katalogu co ten notebook.


```python
from fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    "Add two numbers"
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")
```

następnie uruchom:

In [12]:
import asyncio
import nest_asyncio
nest_asyncio.apply()


from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

async def demo_mcp():
    client = MultiServerMCPClient(
        {
            "math": {
                "transport": "stdio",
                "command": "python",
                "args": ["math_server.py"],
            },
        }
    )
    tools = await client.get_tools()
    agent = create_agent("gpt-5-mini", tools)

    r1 = await agent.ainvoke({"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]})
    print(r1["messages"][-1].content)

# Odkomentuj poniżej, aby uruchomić
# asyncio.run(demo_mcp())

(3 + 5) × 12 = 8 × 12 = 96.
